# ***Imputing Case Outcomes***

Since legalBERT is really hard to train with my current computer power, let's try alternative ML algorithms to predict case outcomes and apply MONTE CARLO estimation of f(P_correct_prediction), so we can factor that while fitting a GLM to the imputed case outcomes.

In [1]:
#importing packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
#loading csv files
cases = pd.read_csv('/content/drive/My Drive/ECO225_Final_Project/PART1/cases.csv')
names = pd.read_csv('/content/drive/My Drive/ECO225_Final_Project/PART1/names.csv')
attorneys = pd.read_csv('/content/drive/My Drive/ECO225_Final_Project/PART1/attorneys.csv')
labelled_case_outcomes = pd.read_csv('/content/drive/My Drive/ECO225_Final_Project/PART1/labelled_case_outcomes.csv')

<ipython-input-28-03c39163892a>:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  names = pd.read_csv('/content/drive/My Drive/ECO225_Final_Project/PART1/names.csv')


In [29]:
print(cases.columns,'\n', names.columns,'\n', attorneys.columns,'\n',
     '\n', patents.columns)

Index(['case_row_id', 'case_number', 'case_number_raw', 'district_id',
       'court_name', 'pacer_id', 'case_name', 'assigned_to', 'referred_to',
       'case_cause', 'jurisdictional_basis', 'date_filed', 'date_closed',
       'date_last_filed', 'jury_demand', 'demand', 'lead_case', 'related_case',
       'settlement', 'case_type_1', 'case_type_2', 'case_type_3',
       'case_type_note'],
      dtype='object') 
 Index(['case_row_id', 'case_number', 'case_number_raw', 'district_id',
       'party_row_count', 'party_type', 'name', 'name_long'],
      dtype='object') 
 Index(['case_row_id', 'case_number', 'case_number_raw', 'district_id',
       'party_row_count', 'party_type', 'attorney_row_count', 'attorney_name',
       'attorney_contactinfo', 'position'],
      dtype='object') 
 
 Index(['case_row_id', 'case_number', 'district_id', 'nos', 'date_filed',
       'case_name', 'case_type_1', 'case_type_2', 'case_type_3',
       'case_type_note', 'patent', 'patent_doc_type'],
      dtype='

In [49]:
#isolating useful features to predict case outcomes
#had to leave out some useful features due to many NaN values : (
clean_cases = pd.DataFrame(cases[['case_row_id', 'case_number', 'pacer_id',
                                 'district_id', 'court_name', 'assigned_to',
                                 'referred_to', 'case_cause',
                                 'jurisdictional_basis', 'date_filed',
                                 'date_closed', 'jury_demand','case_type_1']])

#created a new columne to combine assigned_to and referred_to
clean_cases['Judge'] = clean_cases['referred_to'].fillna(clean_cases[
                                                              'assigned_to'])

clean_names = pd.DataFrame(names[['case_row_id', 'case_number', 'party_type',
                                  'name', 'name_long']])

clean_attorneys = pd.DataFrame(attorneys[['case_row_id', 'case_number',
                                          'party_type', 'attorney_name',
                                          'position']])

clean_outcomes = pd.DataFrame(labelled_case_outcomes[['case_row_id',
                                                      'case_number',
                                                      'Plaintiff']])


In [52]:
#adding in the case outcome variable into the cleaned dfs
cases_and_outcomes = pd.merge(clean_cases, clean_outcomes, on = 'case_row_id')

names_and_outcomes = pd.merge(clean_names, clean_outcomes, on = 'case_row_id')

attorneys_and_outcomes = pd.merge(clean_attorneys, clean_outcomes,on = 'case_row_id')

In [55]:
len(clean_outcomes)

6830